# Problem
In this report we are going to solve a optimization problem which is to assign n tasks to n agents and assigning each job to each agent has a given cost. Our task was to minimize this cost using Ant Colony Optimization.

In this report I will explain my code and also the effect of each parameter on the final result.

# Turning Problem Into TSP
Our first step is to change this problem into a TSP so that we can solve it with TSP. I found out that some important points must be considered in order to do this successfully: 


*   In this problem there is no need to create a loop and we just have to calculate the cost of each city with its corresponding index 
*   For reading the cost table it doesn't matter what the previous city was, rather which city is in which index
* While updating the pheromone table , if we updated the $i^{th}$ row and $j^{th}$ column , we must not update the $j^{th}$ row and $i^{th}$ column as well

The reason for these changes is that in this problem it does not matter which city came after or before which city rather than which agent is doing which job. (i.e, which city is in which index)

 



# ACO Algorithm

First I will give a general overview of what my code does and then evaluate the parameters.

***Ants:***

In my code I have an Ant class that creates an instance for each ant and stores it's information, such as memory of which cities it's been to, fitness of it's path so far and the city that it's currently in. In each generation I will create m ants and randomly scatter them in n cities.

***Move Ants***

In each generation of ants we will move all ants at once. Which means we will iterate all the ants n times and in each iteration we move each ant once and take it to the next city. For choosing the next city we will use The Random Proportional using the given $α$ and $β$  parameters. And then give the possible choices and their properties to the choices function in random library. After choosing the next city and moving our ant we will add its fitness to our ant's fitness.

***Phermonos***

After The Ants were moved each of them left a certain amount of phromon in the option they picked which will later be used for other ants to determine if it was a good path or not.
The amount of this pheromone is determined using the given cost table for each testcase and also the constant pheromone parameter.


***Evaporation***

After all ants visited all n cities we will evaporate the phoromones in order to reduce the chance of repeating the more costly paths.
But every time we will evaporate the path of the best ant a bit less in order to make it more likely to be chosen in the future. 

***Generations***

When all of the ants visited all cities and left their phromones, We will remove them and create a new set of ants randomly scattered all over the cities that use the pheromone table created by the previous generations and repeat the process until the termination condition occurs.

In [1]:
import random
import math


In [2]:
class Ant:
  def __init__(self, city):
    self.memory = []
    self.fitness = 0
    self.city = int(city)


In [7]:
class ACO:
  def __init__(self, city_number, ant_number, alpha, beta, phro_amount, evap_rate):
    self.ant_number = ant_number
    self.alpha = alpha
    self.beta = beta
    self.all_cities = [i for i in range(city_number)]
    self.duration = [[0 for i in range(city_number)] for j in range(city_number)]
    self.pheromons = [[1 for i in range(city_number)] for j in range(city_number)]
    self.ants = []
    self.best_result = []
    self.best_fitness = 1000 * 1000 * 1000
    self.pher_amount = phro_amount
    self.evap_rate = evap_rate
    self.city_number = city_number
  
  def formula(self, ant: Ant, city):
    return self.pheromons[city][len(ant.memory)]**self.alpha * ((1/self.duration[city][len(ant.memory)])**self.beta)

  def choose_next_city(self, ant: Ant):
    sum = 0
    remaining_cities = list(set(self.all_cities)- set(ant.memory))
    for city in remaining_cities:
      sum += self.formula(ant, city)
    possibilities = []
    for city in remaining_cities:
      possibilities.append((self.formula(ant, city))/sum)
    choices = random.choices(population=remaining_cities, weights=possibilities, k=1)
    return int(choices[0])

  def moving_the_ant(self, ant: Ant):
    next_city = self.choose_next_city(ant)
    self.pheromons[next_city][len(ant.memory)] += self.pher_amount/self.duration[next_city][len(ant.memory)]
    ant.fitness += self.calculate_fitness(ant, next_city)
    ant.city = next_city
    ant.memory.append(next_city)

  def calculate_fitness(self, ant:Ant, next_city):
    return self.duration[next_city][len(ant.memory)]

  def evaporation_process(self, best_ant:Ant):
    for i in range(self.city_number):
      for j in range(self.city_number):
        if best_ant.memory[j] == i:
          self.pheromons[i][j] = (1-(self.evap_rate/2))*self.pheromons[i][j]
          self.pheromons[i][j] = max(self.pheromons[i][j], 0.2)
        else:
          self.pheromons[i][j] = (1-self.evap_rate)*self.pheromons[i][j]
          self.pheromons[i][j] = max(self.pheromons[i][j], 0.2)
        

  def get_starting_data(self, file_name):
    f = open(file_name, "r")
    txt = f.readlines()

    for i in range(self.city_number):
      durations_list = [float(j) for j in txt[i].split()]
      for j in range(len(durations_list)):
          self.duration[i][j] = durations_list[j]

    
  def create_ant_colony(self):
    self.ants = []
    for i in range(self.ant_number):
      first_city = random.randint(0, (self.city_number-1))
      ant = Ant(city=first_city)
      ant.memory.append(first_city)
      ant.fitness = self.duration[first_city][0]
      self.ants.append(ant)


  def start_ACO_algorithm(self, file_name, generations):
    for i in range(generations):
      self.get_starting_data(file_name=file_name)
      self.create_ant_colony()
      self.each_generation()

    print(self.best_fitness)
    print(self.best_result)

  def each_generation(self):
    # print(self.city_number)
    for i in range(self.city_number-1):
      for j in range(self.ant_number):
        self.moving_the_ant(ant=self.ants[j])

    best_ant = self.ants[0]
    for i in range(self.ant_number):
        if self.ants[i].fitness < self.best_fitness:
            self.best_fitness = self.ants[i].fitness
            self.best_result = self.ants[i].memory
            best_ant = self.ants[i]
    
    self.evaporation_process(best_ant)

# Test Cases

In [8]:
job_1_result = ACO(city_number=4, ant_number=10, alpha=2, beta=1, phro_amount=10, evap_rate=0.8)
job_1_result.start_ACO_algorithm(file_name="job1.assign", generations=4)

26.0
[3, 1, 2, 0]


In [ ]:
job_2_result = ACO(city_number=100, ant_number=10, alpha=2, beta=1, phro_amount=10, evap_rate=0.9)
job_2_result.start_ACO_algorithm(file_name="job2.assign", generations=100)

348.0
[12, 53, 2, 46, 27, 24, 95, 34, 51, 30, 16, 1, 44, 38, 70, 98, 89, 32, 41, 25, 31, 10, 54, 17, 49, 87, 19, 36, 75, 37, 11, 22, 50, 78, 5, 18, 52, 60, 84, 86, 4, 56, 77, 61, 96, 80, 29, 64, 43, 8, 47, 74, 72, 33, 97, 90, 7, 93, 91, 63, 92, 68, 67, 3, 26, 65, 62, 0, 14, 21, 81, 39, 79, 66, 45, 58, 35, 82, 94, 40, 88, 71, 55, 23, 48, 6, 42, 9, 28, 83, 69, 15, 76, 99, 57, 20, 73, 13, 85, 59]


In [ ]:
job_3_result = ACO(city_number=200, ant_number=10, alpha=0.5, beta=5, phro_amount=10, evap_rate=0.9)
job_3_result.start_ACO_algorithm(file_name="job3.assign", generations=10)

727.0
[143, 160, 133, 27, 79, 138, 176, 171, 0, 22, 86, 17, 34, 56, 100, 163, 31, 140, 46, 152, 172, 8, 148, 95, 126, 149, 83, 142, 136, 110, 36, 71, 41, 54, 74, 150, 154, 122, 94, 55, 9, 24, 123, 173, 69, 21, 81, 120, 127, 50, 84, 194, 67, 180, 168, 15, 43, 135, 45, 14, 77, 102, 64, 174, 72, 16, 101, 26, 11, 153, 175, 130, 35, 20, 190, 78, 182, 139, 3, 62, 98, 119, 162, 141, 112, 61, 76, 29, 115, 1, 53, 178, 48, 60, 28, 170, 191, 198, 179, 97, 187, 96, 63, 70, 5, 137, 23, 89, 193, 131, 10, 58, 197, 111, 68, 113, 92, 134, 158, 189, 19, 166, 6, 181, 30, 147, 124, 161, 57, 93, 91, 177, 51, 117, 151, 145, 104, 144, 108, 157, 52, 156, 37, 129, 183, 25, 155, 167, 105, 132, 118, 99, 66, 103, 38, 195, 82, 159, 12, 40, 47, 185, 128, 75, 73, 65, 146, 49, 186, 18, 39, 116, 169, 196, 4, 80, 109, 165, 59, 114, 7, 33, 192, 42, 2, 184, 125, 85, 13, 199, 87, 121, 164, 88, 32, 44, 188, 106, 90, 107]


# Parameter evaluation
***evap_rate***

This parameter determines the amount of evaporation.In the second test case I tried different numbers and found out that if the evaporation is too much (evap_rate to small) It won't result in a good result which meant that the good path would be forgotten but also that after evap_rate of 0.8/0.9 it will again start generating more costly results.I also decided to eliminate any pheromone less than 0.2/0.1 and replace it with 0.2/0.1 in order to let the paths be chosen again and give the option more chances.


***alpha and beta***

The alpha and beta parameters were dependent on each other. I realised that in my algorithm if the alpha param is greater than beta
it will result in a more costly result. And also the params shouldn't be too big or too small. I realized that (a=0.5, b=5) for bigger test cases and (a=1, b=2) for smaller test cases gave the best result

***Iterations and speed***

In my algorithm after at most 100 iterations the code stopped creating better results and with 100 iterations it took 30-40 minutes for the 4th test case to generate a result less than 22k.

# Extras

Test cases for extra score:

In [ ]:
job_4_result = ACO(city_number=1000, ant_number=20, alpha=0.5, beta=10, phro_amount=5, evap_rate=0.9)
job_4_result.start_ACO_algorithm(file_name="job4.assign", generations=100)

21302.0
[472, 99, 381, 509, 878, 98, 175, 97, 224, 633, 703, 548, 569, 994, 778, 756, 540, 877, 200, 363, 895, 462, 5, 380, 726, 558, 654, 355, 881, 402, 123, 397, 944, 792, 387, 324, 375, 716, 116, 684, 516, 768, 529, 438, 459, 490, 520, 195, 235, 107, 334, 328, 174, 207, 212, 572, 48, 506, 600, 351, 842, 523, 487, 53, 239, 248, 421, 230, 535, 268, 992, 170, 851, 152, 321, 957, 990, 416, 445, 928, 765, 498, 289, 817, 991, 488, 446, 127, 182, 17, 513, 134, 682, 925, 126, 0, 188, 793, 904, 47, 730, 278, 9, 467, 660, 64, 766, 352, 342, 769, 888, 39, 972, 106, 261, 495, 437, 393, 75, 430, 640, 622, 818, 203, 419, 610, 767, 854, 149, 412, 623, 299, 404, 764, 636, 903, 316, 865, 366, 681, 196, 570, 238, 518, 257, 82, 474, 339, 712, 6, 782, 983, 580, 694, 218, 915, 863, 37, 874, 384, 604, 545, 69, 85, 626, 201, 948, 586, 410, 455, 659, 105, 466, 30, 563, 19, 658, 307, 209, 79, 665, 701, 426, 422, 413, 96, 827, 714, 350, 749, 197, 138, 678, 582, 649, 11, 930, 503, 471, 29, 267, 89, 161, 544, 

In [ ]:
job_5_result = ACO(city_number=2000, ant_number=20, alpha=0.5, beta=10, phro_amount=5, evap_rate=0.9)
job_5_result.start_ACO_algorithm(file_name="job5.assign", generations=100)

48272.0
[856, 1510, 1929, 1782, 741, 290, 528, 1635, 238, 600, 424, 1661, 41, 637, 1793, 1471, 1413, 171, 408, 517, 753, 1157, 318, 157, 1251, 1756, 1098, 598, 1846, 1291, 1844, 1370, 1733, 228, 795, 1312, 728, 97, 241, 660, 260, 302, 1921, 431, 917, 1546, 355, 1486, 1439, 72, 1422, 1447, 847, 1099, 1652, 1865, 575, 330, 81, 1146, 543, 1554, 1794, 229, 1451, 830, 1592, 852, 1477, 331, 1902, 1508, 1066, 467, 1347, 482, 996, 895, 1649, 1532, 163, 379, 1310, 662, 523, 1892, 1141, 1201, 652, 1231, 1613, 1854, 279, 829, 842, 353, 1962, 350, 1302, 1537, 38, 804, 563, 1420, 1040, 488, 1820, 438, 1668, 1197, 596, 1976, 436, 1285, 1170, 1491, 619, 516, 1587, 1577, 18, 761, 341, 1443, 1611, 251, 813, 1543, 1639, 1594, 1765, 88, 1738, 1032, 796, 230, 1873, 757, 771, 926, 627, 1208, 1323, 612, 403, 799, 1069, 763, 744, 865, 835, 941, 485, 1124, 518, 503, 974, 1428, 805, 54, 843, 1951, 1371, 573, 388, 317, 1916, 697, 1306, 1359, 1203, 1983, 670, 1081, 1901, 1216, 704, 1492, 554, 168, 1108, 1150, 15